In [29]:
import pandas as pd
import json

import re

import utilities 

import networkx as nx

from itertools import combinations, product


In [30]:
G = nx.MultiDiGraph()

In [31]:
# open neighborhoods json file

with open('dicts/neighborhood_polys.json','r') as f:
    neighborhoods = json.load(f)
    
properties_neighborhood_aggregated = pd.read_csv('../data/properties_neighborhood_aggregated.csv')


In [32]:
# each neighborhood is a node

for i in range(len(list(neighborhoods.keys()))):
    G.add_node(list(neighborhoods.keys())[i], neighborhood={'name':list(neighborhoods.keys())[i]})
    
for i in range(len(properties_neighborhood_aggregated)):
    G.nodes()[properties_neighborhood_aggregated['neighborhood'].iloc[i]]['neighborhood'].update({"avg_property_value":properties_neighborhood_aggregated['unit_zestimate'].iloc[i]})


In [33]:
# 'is_next_to' edge relationship for neighborhood

hoods = list(G.nodes())
hood_combs = list(combinations(hoods, 2))

for i in range(len(hood_combs)):
  
    if utilities.intersection(neighborhoods, hood_combs[i][0], neighborhoods, hood_combs[i][1]):
        G.add_edge(hood_combs[i][0],hood_combs[i][1], NEXT_TO={}) # relationship is...
        G.add_edge(hood_combs[i][1],hood_combs[i][0], NEXT_TO={}) # bidirectional
            

TopologyException: side location conflict at -87.836580878737252 41.986396112425794
TopologyException: side location conflict at -87.664020791475323 41.947275771403895
TopologyException: side location conflict at -87.648784708051664 41.939987536474959
TopologyException: side location conflict at -87.624576496994393 41.896975362507106
TopologyException: side location conflict at -87.780022286293374 41.997413553897694
TopologyException: side location conflict at -87.780022286293374 41.997413553897694


In [49]:
list(G.nodes(data=True))

[('Grand Boulevard',
  {'neighborhood': {'name': 'Grand Boulevard',
    'avg_property_value': 462.85965830728884}}),
 ('Printers Row',
  {'neighborhood': {'name': 'Printers Row',
    'avg_property_value': 273.67922443502806}}),
 ('United Center',
  {'neighborhood': {'name': 'United Center',
    'avg_property_value': 231.33038583952887}}),
 ('Sheffield & DePaul',
  {'neighborhood': {'name': 'Sheffield & DePaul',
    'avg_property_value': 5792.703159256571}}),
 ('Humboldt Park',
  {'neighborhood': {'name': 'Humboldt Park',
    'avg_property_value': 3507.0095553309943}}),
 ('Garfield Park',
  {'neighborhood': {'name': 'Garfield Park',
    'avg_property_value': 1834.928963401752}}),
 ('North Lawndale',
  {'neighborhood': {'name': 'North Lawndale',
    'avg_property_value': 2092.5473552368617}}),
 ('Little Village',
  {'neighborhood': {'name': 'Little Village',
    'avg_property_value': 1550.6294410436078}}),
 ('Armour Square',
  {'neighborhood': {'name': 'Armour Square',
    'avg_property_

In [35]:
# save to object

neo_neighborhoods = utilities.nx_to_neo_nodes(G, return_nodes=True, return_edges=True)

IndexError: list index out of range

In [9]:
nx_nodes = list(G.nodes(data=True))

asdf = "CREATE " + "(" + re.sub(r'\W+','', nx_nodes[0][0]) + ":" + list(nx_nodes[0][1].keys())[0] + " {" + list(list(nx_nodes[0][1].values())[0].keys())[0] + ":" + '"' + list(nx_nodes[0][1].values())[0]['name'] + '"' + "," + list(list(nx_nodes[0][1].values())[0].keys())[1] + ":" + '"' + str(list(nx_nodes[0][1].values())[0]['avg_property_value']) + '"' + "}" + ")"




'CREATE (GrandBoulevard:neighborhood {name:"Grand Boulevard",avg_property_value:"462.85965830728884"})'

In [442]:
# reset the graph

G = nx.MultiDiGraph()

In [443]:
# opening census tracts json file

with open('dicts/tract_polys.json','r') as f:
    tracts = json.load(f)


In [444]:
# each tract is a node

for i in range(len(list(tracts.keys()))):
    G.add_node(list(tracts.keys())[i], tract={'name':list(tracts.keys())[i]})
    


In [445]:
# 'is_next_to' edge relationship for tracts

tract_names = list(G.nodes())
tract_combs = list(combinations(tract_names, 2))

for i in range(len(tract_combs)):
  
    if utilities.intersection(tracts, tract_combs[i][0], tracts, tract_combs[i][1]):
        G.add_edge(tract_combs[i][0],tract_combs[i][1], NEXT_TO={}) # relationship is...
        G.add_edge(tract_combs[i][1],tract_combs[i][0], NEXT_TO={}) # bidirectional
            

In [446]:
neo_tracts = nx_to_neo_nodes(G, return_nodes=True, return_edges=True)

In [447]:
# reset the graph

G = nx.MultiDiGraph()

In [448]:
# each neighborhood is a node

for i in range(len(list(neighborhoods.keys()))):
    G.add_node(list(neighborhoods.keys())[i], neighborhood={'name':list(neighborhoods.keys())[i]})
    


In [449]:
# each tract is a node

for i in range(len(list(tracts.keys()))):
    G.add_node(list(tracts.keys())[i], tract={'name':list(tracts.keys())[i]})
    


In [450]:
# 'is_within' and 'contains' edge relationship for tract -> neighborhood

hood_names = list(neighborhoods.keys())
tract_names = list(tracts.keys())

combs = list(product(hood_names, tract_names))

for i in range(len(combs)):
  
    if utilities.intersection(neighborhoods, combs[i][0], tracts, combs[i][1]):
        G.add_edge(combs[i][0],combs[i][1], CONTAINS={}) # relationship is...
        G.add_edge(combs[i][1],combs[i][0], IS_WITHIN={}) # bidirectional
            

In [451]:
neo_contain_relationships = nx_to_neo_nodes(G, return_nodes=False, return_edges=True)

In [452]:
to_neo = neo_neighborhoods + neo_tracts + neo_contain_relationships


In [453]:
# save as txt file

with open('neo.txt', 'w') as neo_text:
    for listitem in to_neo:
        neo_text.write('%s\n' % listitem)